In [1]:
pip install pandas numpy

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import ast

# Load raw data files
users_df = pd.read_csv('../RAW/raw_users.csv')
training_sessions_df = pd.read_csv('../RAW/raw_training_sessions.csv')

# 1. Data Validation and Cleaning
def validate_and_clean_users(df):
    # Check for missing columns
    expected_columns = ['_id', 'username', 'email', 'password', 'role', 'designation']
    missing_columns = [col for col in expected_columns if col not in df.columns]
    if missing_columns:
        print(f"Missing columns in users data: {missing_columns}")
    
    # Fill missing values with appropriate defaults
    df['username'].fillna('', inplace=True)
    df['email'].fillna('', inplace=True)
    df['password'].fillna('', inplace=True)
    df['designation'].fillna('', inplace=True)
    
    # Remove duplicates
    df.drop_duplicates(inplace=True)
    
    return df

def validate_and_clean_training_sessions(df):
    # Check for missing columns
    expected_columns = [ 'training_code', 'status', 'trainer','username', 'startDate', 'endDate', 'participants']
    missing_columns = [col for col in expected_columns if col not in df.columns]
    if missing_columns:
        print(f"Missing columns in training sessions data: {missing_columns}")
    
    # Convert date columns to datetime format
    df['startDate'] = pd.to_datetime(df['startDate'], errors='coerce')
    df['endDate'] = pd.to_datetime(df['endDate'], errors='coerce')
    
    # Parse participants column as a list of dictionaries
    df['participants'] = df['participants'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else [])
    
    # Remove duplicates, excluding 'participants' column
    df.drop_duplicates(subset=[col for col in df.columns if col != 'participants'], inplace=True)
    
    return df

# Apply validation and cleaning functions
users_df = validate_and_clean_users(users_df)
training_sessions_df = validate_and_clean_training_sessions(training_sessions_df)

# 2. Staging Layer Preparation

# Fill missing scores in participants
def fill_missing_scores(participants):
    # Calculate averages for each score across all participants in this session
    avg_scores = {
        'hackerRankScore': np.nanmean([p.get('hackerRankScore', np.nan) for p in participants]),
        'assessmentScore': np.nanmean([p.get('assessmentScore', np.nan) for p in participants]),
        'performance': np.nanmean([p.get('performance', np.nan) for p in participants]),
        'communication': np.nanmean([p.get('communication', np.nan) for p in participants])
    }
    
    # Fill in missing values
    for participant in participants:
        participant['hackerRankScore'] = participant.get('hackerRankScore', avg_scores['hackerRankScore'])
        participant['assessmentScore'] = participant.get('assessmentScore', avg_scores['assessmentScore'])
        participant['performance'] = participant.get('performance', avg_scores['performance'])
        participant['communication'] = participant.get('communication', avg_scores['communication'])
    
    return participants

# Apply the function to each participants list in the DataFrame
training_sessions_df['participants'] = training_sessions_df['participants'].apply(fill_missing_scores)

# Explode participants into individual rows
staging_sessions_df = training_sessions_df.explode('participants').reset_index(drop=True)

# Normalize participants into separate columns
participants_df = pd.json_normalize(staging_sessions_df['participants']).add_prefix('participant_')

# Concatenate normalized participants data with the main staging DataFrame
staging_sessions_df = pd.concat([staging_sessions_df.drop(columns=['participants']), participants_df], axis=1)

# 3. Save the staging data to CSV files
users_df.to_csv('staging_users.csv', index=False)
staging_sessions_df.to_csv('staging_training_sessions.csv', index=False)

print("Validation, cleaning, and staging data preparation completed. Files saved as 'staging_users.csv' and 'staging_training_sessions.csv'.")


Missing columns in training sessions data: ['username']
Validation, cleaning, and staging data preparation completed. Files saved as 'staging_users.csv' and 'staging_training_sessions.csv'.


C:\Users\LovishMalhotra\AppData\Local\Temp\ipykernel_2848\1487035109.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['username'].fillna('', inplace=True)
C:\Users\LovishMalhotra\AppData\Local\Temp\ipykernel_2848\1487035109.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.